<a href="https://colab.research.google.com/github/Jailsonrs/Classificacao-Binaria-AMNZN/blob/main/Analise_amnzn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Outline

* Introdução
* Metodologia
  * Descrição dos procedimentos
*Aplicação
  * Análise Descritiva e Limpeza
  * Treinamento dos modelos 
    * glmnet
    * Random Forest
    * Random Forest com Oversampling
* Resultados e conclusões
    * Estratégia de monitoramento e deploy




---



#Introdução

Nesse notebook ajustaremos dois modelos preditivos de classificação binária.
 
Começamereos com uma elastic net e em seguida um Random Forest. Por fim, treinaremos uma Random Forest com sobreamostragem e verificaremos o impacto na performance dos modelos.
 
O modelo preditivo escolhido deverá ser capaz de classificar a cada dia a probabilidade de que a ação da empresa ***AMZN*** negociada na Nasdaq alcance nos próximos 10 dias uma valorização de no
mínimo 4% e que não desvalorize mais que 2,5%.
 
Como o objetivo do modelo é alcançar uma boa capacidade preditiva para os critérios acima, não focaremos tanto nos preditores lineares e suas propriedades estatísticas e nem em interpretabilidade. Portanto, tomaremos uma abordagem focada em ***Machine Learning***.
 
 
 
---

#Metodologia

Primeiro foi conduzida uma Análise exploratória de dados e foram removidas as variáveis com pouca ou nenhuma variabilidade. Variáveis infladas de zero também foram removidas pois podem causar sobredispersão e diminuir a precisão dos modelos.

O dataset foi higienizado e as variáveis foram 'parseadas' para seus tipos adequados. 

Procurou-se reduzir a dimensionalidade o máximo possivel para não tornar o modelo super parametrizado, assim evitando overfitting durante o treinamento. 

Os modelos utilizados foram Regressão logística regularizada e Random Forest (RF). No RF foi treinada uma variante utilizando o método SMOTE para dados desbalanceados.

O método de grid search foi implementado tendo em vista a redução da possibilidade de overfitting. 

Utilizou-se também n-fold crossvalidation, com $n=6$.
Foram utilzados 6 folds, pois esse valor mantém aproximadamente a proporção 75/25 para treino e teste durante o treinamento.

Por fim, os resultados dos modelos foram comparados utilizando diversas metricas que levassem em conta o desbalanceamento das classes, comparando com métricas padrão e decidiu-se utilizar o que obteve maior performance.




---



#Aplicação

Dependências necessárias para rodar o notebook passo a passo:

In [ ]:
library(dplyr) 
library('data.table')
library(devtools)
library(purrr)
library('caret')
library('e1071')
library('glmnet')
library('DMwR')
library("Hmisc")
library('MLmetrics')
library("knitr")
library('kableExtra')

options(warnings=-1,warn=-1)

### *Análise descritiva e limpeza*






*   ***Leitura dos dados***





In [ ]:
##reading dataset
setwd("/")
data = data.table::fread('/content/sample_data/AMZN ATUALIZADO.csv', 
                            sep=';', dec = ',')
##Renaming response levelsinstall.packages
data$Variavel_resposta <- ifelse(data$Variavel_resposta == 1,
                                                    'Sim',
                                                    'Nao')



*   ***Dados faltantes***





Depois de uma breve análise da distribuição de dados faltantes, escolheu-se removê-los por questão de simplicidade. Haviam 30 observações faltantes na variável resposta.

In [ ]:
head(data,3)

date,ticker,ALPHA_OVERRIDABLE,ASSET_CVRG_RATIO,BB_1YR_DEFAULT_PROB,BB_2Y_DEFAULT_PROB,BB_3Y_DEFAULT_PROB,BB_4Y_DEFAULT_PROB,BB_5Y_DEFAULT_PROB,BEST_ANALYST_RATING,⋯,VAR_SWAP_12M_LV,VOLATILITY_20D,VOLATILITY_260D,VOLATILITY_30D,VOLATILITY_360D,VOLATILITY_90D,VWAP_VOLUME,WACC_COST_DEBT,WACC_COST_EQUITY,Variavel_resposta
<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<fct>
12/02/2010 00:00,AMZN US Equity,0.1841,"13,6285",3.0138e-05,0.000721139,0.002394981,0.004595436,0.007077496,4.306,⋯,0,33.915,45.736,31.347,62.273,51.194,8024738,3.2031,10.4506,Nao
16/02/2010 00:00,AMZN US Equity,0.1837,"13,6285",3.1043e-05,0.000732654,0.002420904,0.004634488,0.007125953,4.306,⋯,0,34.224,45.577,31.623,62.226,51.053,8897440,3.2031,10.4506,Sim
17/02/2010 00:00,AMZN US Equity,0.1804,"13,6285",3.1587e-05,0.000739574,0.002436533,0.004658125,0.007155382,4.306,⋯,0,34.085,45.485,31.503,62.218,51.066,8902406,3.2031,10.4506,Sim


In [ ]:
data %>% group_by(Variavel_resposta) %>%
         summarise(count = n(), .groups = 'drop') %>% 
         kable(format = 'markdown', 
         caption = 'Contagem de valores da variável resposta')



Table: Contagem de valores da variável resposta

|Variavel_resposta | count|
|:-----------------|-----:|
|Nao               |  1909|
|Sim               |   944|
|NA                |    34|

***Obs: As quatro observações NA adicionais são linhas vazias que foram lidas junto com o dataset.***

In [ ]:
##removendo  NA's 
data$Variavel_resposta <- as.factor(data$Variavel_resposta);data <- na.omit(data)

In [ ]:
##recontando frequencia nas classes
data %>% group_by(Variavel_resposta) %>%
         summarise(count = n(), .groups = 'drop') %>% 
         kable(format = 'markdown', 
         caption = 'Contagem de valores da variável resposta' )



Table: Contagem de valores da variável resposta

|Variavel_resposta | count|
|:-----------------|-----:|
|Nao               |  1909|
|Sim               |   944|



---





*   ***Baixa variabilidade nas variáveis***



As variáveis abaixo não possuem riqueza de variabilidade, podendo essas apenas trazer ruído ou causar um sobreajuste no modelo. Portanto, optei por tirá-las.


In [ ]:
select_if(data[,-1], is_character) %>%
        lapply(unique) %>% names() %>% 
                     data.frame () %>%
                     kable(format="markdown",
                     caption = 'Variáveis que serão retiradas do modelo por \n       terem pouca variabilidade')



Table: Variáveis que serão retiradas do modelo por 
       terem pouca variabilidade

|.                              |
|:------------------------------|
|ticker                         |
|ASSET_CVRG_RATIO               |
|CAPITAL_EXPEND                 |
|CAP_EXPEND_RATIO               |
|CASH_RATIO                     |
|CFO_TO_AVG_CURRENT_LIABILITIES |
|CF_CASH_FROM_OPER              |
|CF_FREE_CASH_FLOW              |
|CUR_RATIO                      |
|EARN_FOR_COMMON                |
|EBITDA                         |
|EBITDA_TO_INTEREST_EXPN        |
|EBITDA_TO_REVENUE              |
|EBIT_TO_INT_EXP                |
|ENTERPRISE_VALUE               |
|EV_TO_T12M_SALES               |
|FCF_TO_TOTAL_DEBT              |
|GROSS_MARGIN                   |
|GROSS_PROFIT                   |
|HISTORICAL_MARKET_CAP          |
|IS_DIL_EPS_CONT_OPS            |
|NET_DEBT                       |
|NET_DEBT_TO_CASHFLOW           |
|NET_DEBT_TO_EBITDA             |
|NET_INCOME_TO_COMMON_MARGIN


Como  não estamos levando em consideração a dependência temporal dos dados, mas sim a ocorrência de um evento determinado pelos critérios citados na introdução, a coluna de Data também não será necessária.



---





*   ***Dados desbalanceados***






Abaixo, podemos observar que os dados estão um pouco desbalanceados, podendo gerar diversos problemas de precisão e avaliação no nosso modelo.

Temos uma proporção de 33% na classe Positiva, enquanto que na classe negativa temos 67% do total das observações.

Dados desbalanceados são geralmente ocasionados por um plano amostral ruim, que acaba inserindo algum tipo de 'viés' no processo de amostragem, consequentemente retirando as boas propriedades dos estimadores (não enviesado, consistência e completude)

No mundo do IOT, como o processo de coleta de dados é automatizado (eventos em streaming de dados), esse problema pode ocasionado por um processo de mensuração errôneo, que pode ser análogo a um plano amostral ruim.

Podemos contornar esse problema com algoritmos de sobreamostragem ou subamostragem e as métricas comuns de avaliação devem ser vistas com cuidado para nao nos conduzir a erros.

In [ ]:
### proporção nas classes
data %>% group_by(Variavel_resposta,) %>% summarise(count=n(),.groups='drop') %>% 
        mutate(prop = paste0((round(count/sum(count),2)*100),"%"))

Variavel_resposta,count,prop
<fct>,<int>,<chr>
Nao,1909,67%
Sim,944,33%




---





*   ***Variáveis inflacionadas de zeros***




O dataset está quase pronto, faltando apenas remover as variáveis inflacionadas com zeros

In [ ]:
dim(dados) ; select_if(data, is_numeric) %>% tibble() -> dados ; head(dados)

[1] 2853   76

ALPHA_OVERRIDABLE,BB_1YR_DEFAULT_PROB,BB_2Y_DEFAULT_PROB,BB_3Y_DEFAULT_PROB,BB_4Y_DEFAULT_PROB,BB_5Y_DEFAULT_PROB,BEST_ANALYST_RATING,BEST_BPS,BEST_CPS,BEST_CUR_EV_TO_EBITDA,⋯,VAR_SWAP_12M_LV,VOLATILITY_20D,VOLATILITY_260D,VOLATILITY_30D,VOLATILITY_360D,VOLATILITY_90D,VWAP_VOLUME,WACC_COST_DEBT,WACC_COST_EQUITY,Variavel_resposta
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<fct>
0.1841,3.0138e-05,0.000721139,0.002394981,0.004595436,0.007077496,4.306,0,0,0,⋯,0,33.915,45.736,31.347,62.273,51.194,8024738,3.2031,10.4506,Nao
0.1837,3.1043e-05,0.000732654,0.002420904,0.004634488,0.007125953,4.306,0,0,0,⋯,0,34.224,45.577,31.623,62.226,51.053,8897440,3.2031,10.4506,Sim
0.1804,3.1587e-05,0.000739574,0.002436533,0.004658125,0.007155382,4.306,0,0,0,⋯,0,34.085,45.485,31.503,62.218,51.066,8902406,3.2031,10.4506,Sim
0.1816,3.0804e-05,0.000729587,0.002413965,0.004623984,0.007112868,4.306,0,0,0,⋯,0,34.610,45.229,31.787,62.219,51.111,9623213,3.2031,10.4506,Sim
0.1799,3.1048e-05,0.000732714,0.002421041,0.004634696,0.007126212,4.306,0,0,0,⋯,0,31.257,45.234,31.522,62.204,50.949,7031321,3.2031,10.4506,Sim
0.1819,2.9971e-05,0.000718368,0.002387851,0.004583570,0.007061672,4.306,0,0,0,⋯,0,31.184,44.901,30.212,61.653,50.918,6799982,3.2031,10.4506,Sim


Podemos observar que existem colunas em que a ocorrência do valor zero é muito abundante (mais que 65% dos valores de cada variável)

Como a literatura sugere que o excesso de zeros pode ser 
gerado por um mecanismo distinto da fonte original dos dados, iremos remover essas variáveis.

In [ ]:
##Vetor com colunas infladas de zero
colunas_comzeros = c("BEST_ROE"
 ,"BEST_PEG_RATIO"            
 ,"BEST_OPP"                  
 ,"BEST_PE_RATIO_GAAP"        
 ,"BEST_CUR_EV_TO_EBITDA"
 ,"BEST_BPS"                  
 ,"BEST_CPS" 
 ,"BEST_EBIT"                 
 ,"BEST_EBITDA"               
 ,"BEST_EPS"                  
 ,"BEST_EPS_GAAP"             
 ,"BEST_EPS_MEDIAN"           
 ,"BEST_EPS_NUMEST"           
 ,"BEST_NET_DEBT"             
 ,"BEST_NET_INCOME"           
 ,"BEST_SALES")
 
#
for(i in colunas_comzeros){
  dados %>% group_by_at(i) %>% summarise(count = n(),.groups='drop') %>% 
                                  arrange(-count) %>% 
           mutate(prop =  paste0(round(count/sum(count)*100,2),"%")) %>% 
           head(3) %>% 
           knitr::kable(caption = paste("Proporção de zeros na variável",i),
           format="markdown",
           col.names=c(i,
           'Contagem de valores',
           "Proporção"),
            align = "lcr") %>% print        
}    

##ranking (top 3) de valores mais frequentes nas variáveis



Table: Proporção de zeros na variável BEST_ROE

|BEST_ROE | Contagem de valores | Proporção|
|:--------|:-------------------:|---------:|
|0.00     |        2393         |    83.88%|
|29.25    |         60          |      2.1%|
|21.10    |         57          |        2%|


Table: Proporção de zeros na variável BEST_PEG_RATIO

|BEST_PEG_RATIO | Contagem de valores | Proporção|
|:--------------|:-------------------:|---------:|
|0.000          |        2300         |    80.62%|
|1.221          |          5          |     0.18%|
|1.201          |          4          |     0.14%|


Table: Proporção de zeros na variável BEST_OPP

|BEST_OPP | Contagem de valores | Proporção|
|:--------|:-------------------:|---------:|
|0.000    |        2042         |    71.57%|
|2674.000 |         68          |     2.38%|
|3932.667 |         58          |     2.03%|


Table: Proporção de zeros na variável BEST_PE_RATIO_GAAP

|BEST_PE_RATIO_GAAP | Contagem de valores | Proporção|
|:------------------|:--

In [ ]:
## removendo variáveis infladas de zero
dados %>% select(-c("BEST_BPS"                  
 ,"BEST_CPS" 
 ,"BEST_CUR_EV_TO_EBITDA"
 ,"BEST_EBIT"                 
 ,"BEST_EBITDA"               
 ,"BEST_EPS"                  
 ,"BEST_EPS_GAAP"             
 ,"BEST_EPS_MEDIAN"           
 ,"BEST_EPS_NUMEST"           
 ,"BEST_NET_DEBT"             
 ,"BEST_NET_INCOME"           
 ,"BEST_OPP"                  
 ,"BEST_PEG_RATIO"            
 ,"BEST_PE_RATIO_GAAP"        
 ,"BEST_ROE"
 ,"BEST_SALES")) -> dados



---




*   ***Parsing dos dados***



Na próxima célula, convertemos algumas variáveis **int** para **factor** (categórica) pois faz mais sentido por serem discretas e possuirem classes bem definidas.

In [ ]:
## convertendo o tipo de algumas variáveis codificadas para fator.
dados$TOT_SELL_REC <- factor(dados$TOT_SELL_REC)
dados$BLOOMBERG_ISSUER_DEFAULT_SCORE  <- factor(dados$BLOOMBERG_ISSUER_DEFAULT_SCORE )
dados$TOT_HOLD_REC <- factor(dados$BLOOMBERG_ISSUER_DEFAULT_SCORE )
dados$TOT_SELL_REC <- factor(dados$BLOOMBERG_ISSUER_DEFAULT_SCORE )
##removendo as colunas  'NUM_TRADES' e 'PX_OFFICIAL_CLOSE'
dados <- dados[,-which(names(dados) %in% c('NUM_TRADES','PX_OFFICIAL_CLOSE'))]

Agora temos o dataset final, pronto para ser inserido no modelo:

In [ ]:
paste("O dataset final possui",dim(dados)[1],'observações','e',dim(dados)[2],'variáveis.')
head(dados[sample(1:dim(dados)[1],10) ,])

[1] "O dataset final possui 2853 observações e 58 variáveis."

ALPHA_OVERRIDABLE,BB_1YR_DEFAULT_PROB,BB_2Y_DEFAULT_PROB,BB_3Y_DEFAULT_PROB,BB_4Y_DEFAULT_PROB,BB_5Y_DEFAULT_PROB,BEST_ANALYST_RATING,BEST_TARGET_PRICE,BETA_ADJ_OVERRIDABLE,BETA_STD_DEV_ERR_OVERRIDABLE,⋯,VAR_SWAP_12M_LV,VOLATILITY_20D,VOLATILITY_260D,VOLATILITY_30D,VOLATILITY_360D,VOLATILITY_90D,VWAP_VOLUME,WACC_COST_DEBT,WACC_COST_EQUITY,Variavel_resposta
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<fct>
0.0859,1.3600e-05,0.000452287,0.001719844,0.003505322,0.005655800,4.622,653.444,1.1917,0.0984,⋯,39.80606,29.186,33.693,37.271,32.896,33.792,3350224,0.8899,9.6577,Sim
-0.0334,1.0302e-05,0.000377539,0.001504331,0.003129068,0.005137567,4.140,392.056,1.3153,0.1014,⋯,31.18163,23.441,33.462,23.663,31.229,31.876,4603654,0.9915,11.9345,Nao
-0.0284,1.0601e-05,0.000382180,0.001514509,0.003142793,0.005152473,4.163,390.265,1.3031,0.1002,⋯,31.67606,20.147,33.363,21.544,30.239,30.586,2337139,2.4641,12.2127,Nao
0.1375,2.0250e-06,0.000159429,0.000863608,0.002041289,0.003682680,4.222,195.370,0.9648,0.0714,⋯,34.78831,39.947,33.510,33.891,38.797,32.249,5149867,2.4750,11.3154,Sim
0.0836,1.4409e-05,0.000465568,0.001751840,0.003554146,0.005716373,4.674,652.568,1.2046,0.1015,⋯,42.26206,47.478,33.265,39.084,34.241,31.522,2657635,1.0127,9.9689,Sim
0.1439,3.8800e-07,0.000058677,0.000437830,0.001171730,0.002352411,4.769,1655.488,1.0239,0.0925,⋯,34.39238,32.912,22.059,27.702,22.405,29.068,4662142,2.2145,10.8376,Nao




---



### *Split dos dados*



Uma proporção de $80$***%*** do dataset foi utlizada para treino $(2282)$ observações e os $20$***%*** restantes utilizados para validar os modelos após o treinamento.

dataset de treinamento terá $2282$ observações e $58$ variáveis.
Será utilizado durante o treinamento um *6*-*fold* crossvalidation nesse dataset.

O dataset de validação terá $571$ observaçoes e $58$ variáveis.
e será onde testaremos a performance do nossos modelos.




In [ ]:
smp_size = smp_size <- floor(0.80 * nrow(dados))
index = sample(1:dim(dados)[1], size= smp_size)

training =  dados[index, ];
paste("O dataset de treino possui",dim(training)[1],'observações','e',dim(training)[2],'variáveis.')

test = dados[-index, ];
paste("O dataset de validação possui",dim(test)[1],'observações','e',dim(test)[2],'variáveis.')

[1] "O dataset de treino possui 2282 observações e 58 variáveis."

[1] "O dataset de validação possui 571 observações e 58 variáveis."



---



### *Treinamento dos modelos*





Iremos começar com **elasticnet**, um modelo mais simples, porém bastante satisfatório na maioria dos casos. Em seguida testaremos um modelo de **Random Forest**
para termos uma alternativa caso o primeiro falhe.


#### ***(1) Elasticnet***


A função de log-verossimilhança do glmnet é dada a seguir.
Para obter as estimativas do vetor $\pmb{\beta}$ deve-se minimizar a função em relação aos betas.

Perceba que é a função de log-verossimilhanca obtida no modelo de regressão logística comum com dois tipos de regularização ($L_1$ e $L_2$)
 
>$$-\ell(\pmb{\beta} \ | \ \pmb{X})$ = -\frac{1}{N}\sum_{i=1}^N y_i (\beta_0+x_{i}^\intercal\pmb{\beta})-\log(1+e^{(\beta_0+x_{i}^\intercal\pmb{\beta})})+\lambda\left[(1-\alpha)||\pmb{\beta}||_2^2/2+\alpha||\pmb{\beta}||_1^1\right] $$

***Parâmetros e hiperparâmetros do modelo***

Os parâmetros para treinamento do modelo é o vetor:  $$\theta = \pmb{\beta} = \left(\beta_1,\ \beta_2, \ldots, \ \beta_N \right)^\intercal $$ $\hat{\pmb{\beta}}$ pode ser estimado pelo método da máxima verossimilhança, ou seja,     
minimizando $-\ell(\pmb{\beta} \ | \ \pmb{X})$


O vetor de hiperparametros, $\pmb{\eta} = \left(\alpha, \  \lambda \right)^\intercal$ deve ser escolhido utilizando alguma heurística. Aqui utilizaremos o método de busca em grade (grid search), que nada mais é que treinar o modelo utilizando um produto cartesiano de intervalos de valores possíveis pros hiperparâmetros e escolher qual combinação traz o melhor ajuste.


**Aqui,  $\alpha$ controla a importância de cada tipo de regularização no modelo ($L_1$ e $L_2$) e $\lambda$ controla a quantidade de regularização utilizada**

As seguintes restrições são impostas para os hiperparâmetros:

$ \alpha \in \left[0,\ 1 \right]$ e $\lambda > 0$


Utilizando o método **Grid search**, os hiperparâmetros podem ser tunados, de forma a otimizar o modelo

In [ ]:
##produto cartesiano do espaço hiper-paramétrico para realizar grid search
grid_glm <- expand.grid(alpha = seq(0,1,0.1), lambda = seq(0.001,0.3,by = 0.01))

controle = trainControl(method = "cv",
                       number=6,
                       returnResamp="all",
                       classProbs = TRUE,
                       summaryFunction = twoClassSummary,
                       verboseIter = TRUE)

modelo1 <- train(Variavel_resposta~ ., 
                             method = "glmnet", 
                             trControl = controle,
                             metric = "ROC",
                             data = training,
                             tuneGrid =grid_glm)

modelo1

***Resultados do treinamento:***


```
Aggregating results
Selecting tuning parameters
Fitting alpha = 0.1, lambda = 0.001 on full training set
glmnet 

2282 samples
  57 predictor
   2 classes: 'Nao', 'Sim' 

No pre-processing
Resampling: Cross-Validated (6 fold) 
Summary of sample sizes: 1902, 1902, 1901, 1902, 1902, 1901, ...

```



***Matriz de confusão para o modelo***

Embora as classes estejam desbalanceadas, conseguimos obter um modelo com uma acurácia de $0.71$ e um escore ***F1*** igual a $0,81$ que é moderadamente elevado e significativo. 

O teste binomial para acurácia não foi significativo ($p = 0.1142$), assim não temos indícios suficientes para rejeitar a Hipótese nula $$H0:Acc \leq NIR$$ 

Perceba que a *No Information Rate* é muito próxima da *Acurácia*.
significando que se uma pessoa escolher classificar manualmente cada nova observação na classe negativa, ela teria um acerto de $67$% equanto que o modelo em torno de $69$%. 

O ***recall*** do modelo foi de apenas $29$%
o que indica que temos um alto número de falsos negativos.

A **precisão*** foi de $57$%
O que indica um elevado numero de falsos positivos


O escore $$F1 = 0.3916$$ é baixo , o que pode ser considerado ruim, pois $F1$ é uma medida que é usada quando temos classes desbalanceadas, que é nosso o nosso caso.


O **kappa de Cohen** de $0.2143$ é considerado baixo, o que torna o modelo pouco confiável.

Portanto, não temos um bom modelo.

In [ ]:
preditos = predict(modelo1, test[,-58])
confusionMatrix(table(preditos,test$Variavel_resposta),positive='Sim')
paste('Escore F1:',' ',F1_escore(preditos,test$Variavel_resposta))



```
Confusion Matrix and Statistics

        
preditos Nao Sim
     Nao 341 132
     Sim  42  56
                                          
               Accuracy : 0.6953          
                 95% CI : (0.6557, 0.7328)
    No Information Rate : 0.6708          
    P-Value [Acc > NIR] : 0.1142          
                                          
                  Kappa : 0.2143          
                                          
 Mcnemar's Test P-Value : 1.509e-11       
                                          
            Sensitivity : 0.29787         
            Specificity : 0.89034         
         Pos Pred Value : 0.57143         
         Neg Pred Value : 0.72093         
             Prevalence : 0.32925         
         Detection Rate : 0.09807         
   Detection Prevalence : 0.17163         
      Balanced Accuracy : 0.59411         
                                          
       'Positive' Class : Sim             
                                          
'Escore F1:   0.391608391608392'
```



Perceba que o $\phi$ de pearson é muito baixo $(0.23)$. Quanto mais próximo de zero, pior o modelo (ausência de correlação) entre o que foi predito e o que é de fato verdade.  

In [ ]:
##Phi de pearson glmnet
((341*56)-(42*132))/sqrt((341+42)*(132+56)*(341+132)*(42+56))

[1] 0.2345749



---



 
#### ***(2) Random Forest***
 
 
Abaixo iremos treinar um modelo de Random Forest, que nada mais é que um ensemble de classification trees, o que o torna um método versátil (classificação e regressão) e poderoso, devido ao fato de na maioria dos casos sobrepujar modelos lineares.



*   ***Hiperparâmetros do modelo***





In [ ]:
install.packages("ranger")
 
rf_grid <- expand.grid(mtry = c(2, 3, 4, 5),
                      splitrule = c("gini", "extratrees"),
                      min.node.size = c(1, 3, 5))
 
 
cctrl2 <- trainControl(method = "cv", number=6, returnResamp="all",
                       classProbs = TRUE,
                       summaryFunction = twoClassSummary,
                       verboseIter = TRUE)
 
test_class_cv_model2 <- train(Variavel_resposta~ ., method = "ranger", 
                             trControl = cctrl2, metric = "ROC",
                             data = training,
                            # preProc = c("center", "scale"),
                             tuneGrid =rf_grid )
 
test_class_cv_model2

***Resultados do treinamento:***




Abaixo temos o resumo do treinamento do modelo com as iterações do tunning dos hiperparâmetros nas subamostras do cross-validation (6-fold CV)

***Summary of sample sizes*** diz qual o tamanho de cada sub-amostra para onde o conjunto foi treinado.

Os valores dos hiper-parâmetros escolhidos após o grid search foram: 

***mtry = 5, splitrule = extratrees, min.node.size = 1***


```
Random Forest 

2282 samples
  57 predictor
   2 classes: 'Nao', 'Sim' 

No pre-processing
Resampling: Cross-Validated (6 fold) 
Summary of sample sizes: 1902, 1902, 1902, 1901, 1901, 1902, ... 
Resampling results across tuning parameters:

  mtry  splitrule   min.node.size  ROC        Sens       Spec     
  4     extratrees  1              0.9455625  0.9332716  0.7968750
  4     extratrees  3              0.9436413  0.9332716  0.7734375
  4     extratrees  5              0.9394240  0.9312849  0.7604167
  5     gini        1              0.9403510  0.9319515  0.7955729
  5     gini        3              0.9393913  0.9332716  0.7890625
  5     gini        5              0.9384516  0.9306314  0.7838542
  5     extratrees  1              0.9466377  0.9319489  0.7981771
  

ROC was used to select the optimal model using the largest value.
The final values used for the model were mtry = 5, splitrule = extratrees
 and min.node.size = 1. 
```

***Matriz de confusão para o modelo***

In [ ]:
preditos2 = predict(test_class_cv_model2, test[,-58])
confusionMatrix(table(preditos2, test$Variavel_resposta), positive = 'Sim')
paste('Escore F1:',' ',F1_Score(preditos2, test$Variavel_resposta))

A ***acurácia*** geral para o modelo de Random Forest foi de $\bf{0.8809}$, que é $1,26$ vezes maior que a do modelo de Regresão logística regularizado.
 
O teste binomial para a acurácia, dado por:

>$H_0: Acc = NIR$

versus

>$H_1: Acc < NIR$

retornou um ***valor p*** próximo de zero (altamente significativo). Portanto, podemos afirmar que o modelo é bastante confiável e que performa bem em relação a um classificador 'ingênuo' ou sem informação.

Abaixo iremos ver outras estatísticas mais precisas sobre o modelo.



```
Confusion Matrix and Statistics

         
preditos2 Nao Sim
      Nao 355  40
      Sim  28 148
                                          
               Accuracy : 0.8809          
                 95% CI : (0.8515, 0.9063)
    No Information Rate : 0.6708          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.7259          
                                          
 Mcnemar's Test P-Value : 0.1822          
                                          
            Sensitivity : 0.7872          
            Specificity : 0.9269          
         Pos Pred Value : 0.8409          
         Neg Pred Value : 0.8987          
             Prevalence : 0.3292          
         Detection Rate : 0.2592          
   Detection Prevalence : 0.3082          
      Balanced Accuracy : 0.8571          
                                          
       'Positive' Class : Sim             
                                          
'Escore F1:   0.8131'
```



In [ ]:
## Phi de Pearson RF
((355*148)-(28*40))/sqrt((355+28)*(40+148)*(349+40)*(28+148))

[1] 0.7323566

Nesse modelo obtivemos um $\phi$ excelente!
$$\phi= 0.73$$


A estatística ***Kappa de cohen*** dada por $\kappa = \bf{0.7259}$ é excelente, o que significa que o modelo performa bem em relação á um modelo regido por aleatoriedade. Isto é, ela compara as frequencias observadas com as frequencias esperadas

O teste de ***McNemar*** não foi estatísticamente significativo, o que quer dizer que existe hogomeneidade entre as margens da tabela. Isto é, nosso classificador tem elevada concordância com os valores verdadeiros do conjunto de treino

As frequencias esperadas são os valores que qualquer classificador aleatorio esperaria alcançar e as observadas são as obtidas pelo nosso modelo de Random Forest. Nosso classificador apresenta um alto grau de concordância com os valores que foram observado de fato no nosso conjunto de validação.

O ***Recall*** do RF foi de $\bf{78}$***%*** o que indica que ele performa muito bem em relação aos falsos negativos, pois possui poucos. 


A ***Precisão*** do modelo foi de $\bf{84}$***%***. 
Isso quer dizer que de todos os valores preditos como positivos, $\bf{84}$***%***. eram de fato positivos

O escore ***F1*** do modelo é bastante alto ($\bf{0.81}$). Aparentemente o médio desbalanceamento das classes nao afetou o Random Forest.


Uma das vantagens do ***Random Forest*** sobre outros modelos tradicionais de machine learning, é que ele dificilmente sofre sobreajuste (overfitting) e utilizando Cross Validation, essa possibilidade se torna ínfima.



---



#### ***(3) Random Forest com Oversampling***




Aqui utilizaremos uma técnica de oversampling conhecida como SMOTE (*Synthetic Minority Oversampling Technique*) para tentarmos balancear a classe subamostrada e aferir se terá alguma implicação positiva no modelo.
Para isso, adicionaremos o argumento  

```
sampling = 'smote'
```
na função 

```
trainControl
```



In [ ]:
rf_grid3 <- expand.grid(mtry = c(2, 3, 4, 5),
                      splitrule = c("gini", "extratrees"),
                      min.node.size = c(1, 3, 5))


cctrl3 <- trainControl(method = "cv", number=6, 
                       returnResamp="all",
                       classProbs = TRUE,
                       summaryFunction = twoClassSummary,
                       verboseIter = TRUE,
                       sampling = 'smote')

test_class_cv_model3 <- train(Variavel_resposta~ ., method = "ranger", 
                             trControl = cctrl3,
                             metric = "ROC",
                             data = training,
                             tuneGrid =rf_grid )


In [ ]:
test_class_cv_model3 

**Resultados do modelo:**



```
Random Forest 

2282 samples
  57 predictor
   2 classes: 'Nao', 'Sim' 

No pre-processing
Resampling: Cross-Validated (6 fold) 
Summary of sample sizes: 1901, 1902, 1902, 1902, 1901, 1902, ... 
Addtional sampling using SMOTE

Resampling results across tuning parameters:

  mtry  splitrule   min.node.size  ROC        Sens       Spec     
  2     gini        1              0.9298651  0.9351191  0.7513228
  2     gini        3              0.9313598  0.9285678  0.7949735
  5     extratrees  1              0.9420990  0.9338145  0.7857143
  
ROC was used to select the optimal model using the largest value.
The final values used for the model were mtry = 5, splitrule = extratrees
 and min.node.size = 1.
 ```



**Matriz de confusão:**

In [ ]:
preditos3 = predict(test_class_cv_model3, test[,-58])
confusionMatrix(table(preditos3, test$Variavel_resposta), positive = 'Sim')
paste('Escore F1:',' ',F1_escore(preditos3,test$Variavel_resposta))



```
Confusion Matrix and Statistics

         
preditos3 Nao Sim
      Nao 349  40
      Sim  34 148
                                          
               Accuracy : 0.8704          
                 95% CI : (0.8401, 0.8969)
    No Information Rate : 0.6708          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.7042          
                                          
 Mcnemar's Test P-Value : 0.5611          
                                          
            Sensitivity : 0.7872          
            Specificity : 0.9112          
         Pos Pred Value : 0.8132          
         Neg Pred Value : 0.8972          
             Prevalence : 0.3292          
         Detection Rate : 0.2592          
   Detection Prevalence : 0.3187          
      Balanced Accuracy : 0.8492          
                                          
       'Positive' Class : Sim             
                                          
'Escore F1:   0.80'
```



In [ ]:
### Calculando o phi de pearson para nosso RF-SMOTE
((349*148)-(40*34))/sqrt((34+148)*(148+40)*(349+34)*(349+40))

[1] 0.704385



---



#Conclusão

Embora os modelos RF e RF-Smote sejam aparentemente idênticos, Random Forest sem oversampling foi o que melhor perfomou, trazendo evidências em ser mais consistente que os outros.

Escores obtidos em cada modelo:

|  Modlelo/Escore | Acc | kappa |   AUC - ROC | Recall    |Precisão| F1   |$\phi$ de Pearson|
|-----------------|-----|-------|-------------|-----------|--------|------||
| glmnet          |0.71 | 0.22  |     0.7111  |  0.3911   | 0.5578 |0.3981|0.23|
| RF              |0.88 | 0.72  |     0.9466  |  0.7872   | 0.8409 |0.8131|0.73|
| RF-smote        |0.87 | 0.70  |     0.9420  |  0.7872   | 0.8132 |0.8000|0.70|

É possivel melhorar ainda mais utilizando features criadas a partir da combinação linear ou não linear (ou outras funções) das variáveis utilizadas no modelo. A esse processo se dá o nome de ***feature engineering***. No entanto, é preferivel deixar para um trabalho futuro.



*   ***Considerações finais***



Com o modelo testado, o proximo passo é definir uma uma estratégia de monitoramento do modelo
que seja capaz de detectar qualquer drift na distribuição dos dados, evitando assim causar prejuízos ao negócio.
Para isso existem métricas como PSI, K-S entre outras.

Dependendo da infraestrutura do negócio, o deploy do(s) modelo(s) pode ser feito em uma arquitetura 
de microserviços utilizando kubernetes para gerenciar os containers docker onde os modelos rodam. 

É importante também definir um ciclo de integração contínua entre desenvolvimento, teste e produção para 
que haja melhora contínua na capacidade analítica da corporação e da infra estrutura em geral da empresa.





# Apêndice

O coeficiente $\phi$ de pearson é uma métrica muito útil para medir a capacidade preditiva e confiabilidade de um classificador dicotômico cujo dataset de treino seja desbalanceado, pois é uma medida simétrica e não é afetada pelo desbalanceamento das classes. 

Se $\phi = 0$ o modelo não é melhor que o lançamento de uma moeda honesta, se $\phi = -1$ ou $1$ o modelo é ideal.

>$$ \phi = \frac{TP*TN-FP*FN}{\sqrt{(TP+FP)(TP+FN)(TN+FP)(TN+FN)}} $$



---



Função criada para cálculo do escore F1 (Média harmônica entre precisão e recall)

In [ ]:
F1_escore <- function(preditos, teste){
   cm = confusionMatrix(preditos, teste, positive = 'Sim')
   recall = cm$table[4]/(cm$table[4]+cm$table[2])
   precision = cm$table[4]/(cm$table[4]+cm$table[3])
   f1 = 2.000*((recall*precision)/(recall+precision))
   return(f1)
}